ver2 : pred log 변환, xgb parm

In [82]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import kruskal
import scipy

from tqdm import tqdm

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler

from sklearn.linear_model import BayesianRidge, HuberRegressor, Ridge, OrthogonalMatchingPursuit
from sklearn.ensemble import GradientBoostingRegressor
import xgboost
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.linear_model import LogisticRegression


from sklearn.model_selection import KFold, cross_val_score

In [83]:
from sklearn.metrics import make_scorer

def rmsle(y_true, y_pred):
    # 로그변환 후 결측값을 0으로 변환
    log_true = np.nan_to_num(np.log(y_true + 1))
    log_pred = np.nan_to_num(np.log(y_pred + 1))
    
    # RMSLE 계산
    output = np.sqrt(np.mean((log_true - log_pred)**2))
    
    return output

RMSLE = make_scorer(rmsle, greater_is_better=False)

In [84]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [85]:
train_da = pd.read_csv('train_dataset.csv')

In [86]:
train = pd.concat([train, train_da],axis=0)

In [87]:
train

,id,store_sales(in millions),unit_sales(in millions),total_children,num_children_at_home,avg_cars_at home(approx).1,gross_weight,recyclable_package,low_fat,units_per_case,store_sqft,coffee_bar,video_store,salad_bar,prepared_food,florist,cost
0,0.0,8.61,3.0,2.0,2.0,2.0,10.30,1.0,0.0,32.0,36509.0,0.0,0.0,0.0,0.0,0.0,62.09
1,1.0,5.00,2.0,4.0,0.0,3.0,6.66,1.0,0.0,1.0,28206.0,1.0,0.0,0.0,0.0,0.0,121.80
2,2.0,14.08,4.0,0.0,0.0,3.0,21.30,1.0,0.0,26.0,21215.0,1.0,0.0,0.0,0.0,0.0,83.51
3,3.0,4.02,3.0,5.0,0.0,0.0,14.80,0.0,1.0,36.0,21215.0,1.0,0.0,0.0,0.0,0.0,66.78
4,4.0,2.13,3.0,5.0,0.0,3.0,17.00,1.0,1.0,20.0,27694.0,1.0,1.0,1.0,1.0,1.0,111.51
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51358,NaN,3.70,2.0,4.0,4.0,1.0,10.40,1.0,1.0,18.0,34452.0,1.0,0.0,0.0,0.0,1.0,147.17
51359,NaN,10.36,4.0,2.0,0.0,2.0,9.43,0.0,0.0,20.0,30797.0,1.0,1.0,1.0,1.0,1.0,92.57
51360,NaN,11.19,3.0,3.0,0.0,3.0,18.50,0.0,0.0,5.0,30797.0,1.0,1.0,1.0,1.0,1.0,92.57
51361,NaN,3.78,3.0,1.0,1.0,2.0,19.80,0.0,1.0,8.0,23598.0,0.0,0.0,0.0,0.0,1.0,135.92


In [88]:
#train = train.loc[~(train['store_sales(in millions)']>20)]

In [89]:
train.loc[~(train['store_sales(in millions)']>20),'store_sales(in millions)'] = 20

In [90]:
train['unit_sales(in millions)'] = train['unit_sales(in millions)'].replace(6.0, 5.0)
test['unit_sales(in millions)'] = test['unit_sales(in millions)'].replace(6.0, 5.0)

In [91]:
train = train.round(0)

In [92]:
train.columns

Index(['id', 'store_sales(in millions)', 'unit_sales(in millions)',
       'total_children', 'num_children_at_home', 'avg_cars_at home(approx).1',
       'gross_weight', 'recyclable_package', 'low_fat', 'units_per_case',
       'store_sqft', 'coffee_bar', 'video_store', 'salad_bar', 'prepared_food',
       'florist', 'cost'],
      dtype='object')

In [93]:
train = train.drop(['id','recyclable_package','low_fat'], axis=1)
test = test.drop(['id','recyclable_package','low_fat'], axis=1)

In [94]:
def get_values(value):
    return value.values.reshape(-1, 1)

In [95]:
ord_cols = ['store_sales(in millions)','gross_weight']
def scale(train, test):
    for col in train.columns:
        if col in ord_cols:
            print(col)
            scaler = MinMaxScaler()
            train[col] = scaler.fit_transform(get_values(train[col]))
            test[col] = scaler.transform(get_values(test[col]))
    return train, test

In [96]:
train_updated, test_updated = scale(train, test)

store_sales(in millions)
gross_weight


In [97]:
train_updated

,store_sales(in millions),unit_sales(in millions),total_children,num_children_at_home,avg_cars_at home(approx).1,gross_weight,units_per_case,store_sqft,coffee_bar,video_store,salad_bar,prepared_food,florist,cost
0,0.0,3.0,2.0,2.0,2.0,0.2500,32.0,36509.0,0.0,0.0,0.0,0.0,0.0,62.0
1,0.0,2.0,4.0,0.0,3.0,0.0625,1.0,28206.0,1.0,0.0,0.0,0.0,0.0,122.0
2,0.0,4.0,0.0,0.0,3.0,0.9375,26.0,21215.0,1.0,0.0,0.0,0.0,0.0,84.0
3,0.0,3.0,5.0,0.0,0.0,0.5625,36.0,21215.0,1.0,0.0,0.0,0.0,0.0,67.0
4,0.0,3.0,5.0,0.0,3.0,0.6875,20.0,27694.0,1.0,1.0,1.0,1.0,1.0,112.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51358,0.0,2.0,4.0,4.0,1.0,0.2500,18.0,34452.0,1.0,0.0,0.0,0.0,1.0,147.0
51359,0.0,4.0,2.0,0.0,2.0,0.1875,20.0,30797.0,1.0,1.0,1.0,1.0,1.0,93.0
51360,0.0,3.0,3.0,0.0,3.0,0.7500,5.0,30797.0,1.0,1.0,1.0,1.0,1.0,93.0
51361,0.0,3.0,1.0,1.0,2.0,0.8750,8.0,23598.0,0.0,0.0,0.0,0.0,1.0,136.0


florist, prepared_food, video_store, coffee_bar, store_sqft
store_sales(in millions), unit_sales(in millions)


In [98]:
nom_cols = ['unit_sales(in millions)','total_children', 'num_children_at_home', 'avg_cars_at home(approx).1',#'recyclable_package', 'low_fat',
            'units_per_case','store_sqft', 'coffee_bar', 'video_store', 'prepared_food','florist']

In [99]:
nom_cols = [
 'total_children',
 'num_children_at_home',
 'avg_cars_at home(approx).1',
 'store_sqft',
 'coffee_bar',
 'video_store',
 'salad_bar',
 'florist',
]

In [100]:
enc = OneHotEncoder(categories = 'auto', dtype = 'int32', drop = 'first',sparse=False)

In [101]:
nom_train = enc.fit_transform(train_updated[nom_cols])
nom_test = enc.transform(test_updated[nom_cols])

In [102]:
train_update = np.hstack((np.array(train_updated[ord_cols]), nom_train))
test_update = np.hstack((np.array(test_updated[ord_cols]), nom_test))

In [103]:
train_work_sprs =scipy.sparse.hstack([scipy.sparse.coo_matrix(train_update).astype('float64')]).tocsr()
test_work_sprs =scipy.sparse.hstack([scipy.sparse.coo_matrix(test_update).astype('float64')]).tocsr()

In [104]:
best_params = {'n_estimators': 100, # not imp
             'learning_rate': 0.9, # + or - bad
             'max_depth': 15, # - bad + bad
             'lambda': 0.0150, # + bad 
             'alpha': 1.0e-08, # - is little good
             'colsample_bytree': 0.7, # imp (adding randomness to the training process.)
             'min_child_weight': 0, # + bad
             'booster': 'gbtree',
             'grow_policy': 'depthwise', 
             'objective': 'reg:squaredlogerror',
             'eval_metric': 'rmsle',
              }

In [105]:
model = XGBRegressor(**best_params,n_jobs = -1, random_state=42)

classifier = LogisticRegression(random_state=42)
scores = cross_val_score(classifier, train_work_sprs, train.cost, scoring=RMSLE, cv = 3, n_jobs = -1)
classifier.fit(train_work_sprs, train.cost)

# Predict classes for testing set
y_pred = classifier.predict(test_work_sprs)
-np.mean(scores)

In [106]:
scores = cross_val_score(model, train_work_sprs, train.cost, scoring=RMSLE, cv = 3, n_jobs = -1)

In [107]:
-np.mean(scores)

0.2993418522664651

In [108]:
samp_sub = pd.read_csv('sample_submission.csv')

In [109]:
model.fit(train_work_sprs, train.cost)

XGBRegressor(alpha=1e-08, base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.7,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric='rmsle', feature_types=None, gamma=0, gpu_id=-1,
             grow_policy='depthwise', importance_type=None,
             interaction_constraints='', lambda=0.015, learning_rate=0.9,
             max_bin=256, max_cat_threshold=64, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=15, max_leaves=0, min_child_weight=0,
             missing=nan, monotone_constraints='()', n_estimators=100,
             n_jobs=-1, num_parallel_tree=1, ...)

In [110]:
preds = model.predict(test_work_sprs)

In [111]:
samp_sub['cost'] = (preds)
samp_sub.to_csv("submission.csv", index=False)